# Rekko challenge 2019

In [1]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

from pprint import pprint

### Загрузка данных

Замените `DATA_PATH` на путь к данным, которые вы скачали со страницы соревнования.

In [2]:
DATA_PATH = '../../okko/'

---

`catalogue.json` содержит анонимизированную метаинформацию о доступных в сервисе фильмах и сериалах.

In [3]:
with open(os.path.join(DATA_PATH, 'catalogue.json'), 'r') as f:
    catalogue = json.load(f)
    
catalogue = {int(k): v for k, v in catalogue.items()}

In [5]:
len(catalogue)

10200

In [73]:
pprint(catalogue[300])

{'attributes': [6642,
                27158,
                27159,
                27160,
                27161,
                7,
                27162,
                308,
                54,
                14,
                15,
                19,
                20,
                21,
                26640],
 'availability': ['purchase', 'rent', 'subscription'],
 'duration': 90,
 'feature_1': 28027850.538695842,
 'feature_2': 0.688855574,
 'feature_3': 8,
 'feature_4': 1.1265750641,
 'feature_5': 0.6804096966,
 'type': 'movie'}


 - `attributes` — мешок атрибутов
 - `availability` — доступность (может содержать значения `purchase`, `rent` и `subscription`)
 - `duration` — длительность в минутах, округлённая до десятков (продолжительность серии для сериалов и многосерийных фильмов)
 - `feature_1..5` — пять анонимизированных вещественных и порядковых признаков
 - `type` — принимает значения `movie`, `multipart_movie` или `series`

---

`test_users.json` содержит список пользователей, для которых необходимо построить предсказание

In [4]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    print(json.load(f).keys())

dict_keys(['users'])


In [5]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])

In [6]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users_arr = np.array(json.load(f)['users'])

---

`transactions.csv` — список всех транзакций за определённый период времени

In [7]:
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

In [31]:
transactions.head(3)

,element_uid,user_uid,consumption_mode,ts,watched_time,device_type,device_manufacturer
0,3336,5177,S,4.430518e+07,4282,0,50
1,481,593316,S,4.430518e+07,2989,0,11
2,4128,262355,S,4.430518e+07,833,0,50


 - `element_uid` — идентификатор элемента
 - `user_uid` — идентификатор пользователя
 - `consumption_mode` — тип потребления (`P` — покупка, `R` — аренда, `S` — просмотр по подписке)
 - `ts` — время совершения транзакции или начала просмотра в случае просмотра по подписке
 - `watched_time` — число просмотренных по транзакции секунд
 - `device_type` — анонимизированный тип устройства, с которого была совершена транзакция или начат просмотр
 - `device_manufacturer` — анонимизированный производитель устройства, с которого была совершена транзакция или начат просмотр

---

`ratings.csv` содержит информацию о поставленных пользователями оценках

In [8]:
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)

In [35]:
ratings.head(3)

,user_uid,element_uid,rating,ts
0,571252,1364,10,4.430517e+07
1,63140,3037,10,4.430514e+07
2,443817,4363,8,4.430514e+07


 - `rating` — поставленный пользователем рейтинг (от `0` до `10`)

---

`bookmarks.csv` содержит информацию об элементах, добавленных пользователями в список «Избранное»

In [9]:
bookmarks = pd.read_csv(
    os.path.join(DATA_PATH, 'bookmarks.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64
    }
)

In [10]:
bookmarks_us = pd.read_csv(
    os.path.join(DATA_PATH, 'bookmarks.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64
    }, index_col ='user_uid'
)

In [39]:
bookmarks.head(3)

,user_uid,element_uid,ts
0,301135,7185,4.430516e+07
1,301135,4083,4.430516e+07
2,301135,10158,4.430516e+07


### Решение

Для начала построим список элементов, которые тестовые пользователи уже купили или посмотрели по подписке: они не смогут купить их второй раз, а просмотр по подписке второй раз маловероятен, поэтому мы захотим отфильтровать такие элементы из финального ответа.

Точно так же можно поступить и с рейтингами и добавлениями в избранное, если это будет казаться правильным.

In [10]:
from collections import defaultdict

filtered_elements = defaultdict(set)

for user_uid, element_uid in tqdm.tqdm(transactions.loc[:, ['user_uid', 'element_uid']].values):
    if user_uid not in test_users:
        continue
    filtered_elements[user_uid].add(element_uid)

for user_uid, element_uid in tqdm.tqdm(ratings.loc[:, ['user_uid', 'element_uid']].values):
    if user_uid not in test_users:
        continue
    filtered_elements[user_uid].add(element_uid)

100%|██████████████████████████████████████████████████████████████████████| 438790/438790 [00:00<00:00, 757830.50it/s]


In [11]:
filtered_bookmarks = pd.merge(left = bookmarks.loc[:, ['user_uid', 'element_uid']],
                              right = transactions.loc[:, ['user_uid', 'element_uid']])

filtered_bookmarks = pd.DataFrame(filtered_bookmarks.element_uid.values, index=filtered_bookmarks.user_uid, columns=['element_uid'])

---

Для примера мы воспользуемся методом K ближайших соседей, реализованным в библиотеке `implicit`. В качестве данных используем только информацию о рейтингах.

Необходимо построить разреженную матрицу, где строкам будут соответствовать элементы, столбцам — пользователи, а на пересечении пользователя и элемента будет находиться количественная оценка степени их взаимодействия, если таковое имело место.

Не забудем добавить `1` к рейтингу, чтобы избежать деления на ноль во время вычисления `tf-idf`.

In [23]:
ratings['user_uid'] = ratings['user_uid'].astype('category')
ratings['element_uid'] = ratings['element_uid'].astype('category')

ratings_matrix = sp.coo_matrix(
    (ratings['rating'].astype(np.float32) + 1,
        (
            ratings['element_uid'].cat.codes.copy(),
            ratings['user_uid'].cat.codes.copy()
        )
    )
)

ratings_matrix = ratings_matrix.tocsr()

In [14]:
sparsity = ratings_matrix.nnz / (ratings_matrix.shape[0] * ratings_matrix.shape[1])
print('Sparsity: %.6f' % sparsity)

Sparsity: 0.000558


In [24]:
model = TFIDFRecommender()
model.fit(ratings_matrix)

100%|██████████████████████████████████████████████████████████████████████████| 7519/7519 [00:00<00:00, 179163.93it/s]


In [25]:
ratings_matrix_T = ratings_matrix.T.tocsr()

In [15]:
from implicit.nearest_neighbours import TFIDFRecommender

In [12]:
transactions['user_uid'] = transactions['user_uid'].astype('category')
transactions['element_uid'] = transactions['element_uid'].astype('category')
transactions_matrix = sp.coo_matrix(
    (np.ones(transactions.shape[0]),
        (
            transactions['element_uid'].cat.codes.copy(),
            transactions['user_uid'].cat.codes.copy()
        )
    )
)
transactions_matrix = transactions_matrix.tocsr()

Sparcity:

In [13]:
transactions_matrix.nnz / (transactions_matrix.shape[0] * transactions_matrix.shape[1])

0.002326305634915966

In [17]:
transactions_matrix_T = transactions_matrix.T.tocsr()

Обучить модель крайне просто.

In [16]:
model_transactions = TFIDFRecommender()
model_transactions.fit(transactions_matrix)

100%|███████████████████████████████████████████████████████████████████████████| 8296/8296 [00:00<00:00, 17465.12it/s]


bookmarks

In [35]:
bookmarks['user_uid'] = bookmarks['user_uid'].astype('category')
bookmarks['element_uid'] = bookmarks['element_uid'].astype('category')
bookmarks_matrix = sp.coo_matrix(
    (np.ones(bookmarks.shape[0]),
        (
            bookmarks['element_uid'].cat.codes.copy(),
            bookmarks['user_uid'].cat.codes.copy()
        )
    )
)
bookmarks_matrix = bookmarks_matrix.tocsr()

In [36]:
bookmarks_matrix.nnz / (bookmarks_matrix.shape[0] * bookmarks_matrix.shape[1])

0.0006790103524223174

In [42]:
bookmarks_matrix_T = bookmarks_matrix.T.tocsr()

In [37]:
model_bookmarks = TFIDFRecommender()
model_bookmarks.fit(bookmarks_matrix)

100%|██████████████████████████████████████████████████████████████████████████| 9489/9489 [00:00<00:00, 179038.63it/s]


---

Отображения из оригинальной категории во внутреннюю пригодится нам в дальнейшем.

---

В метод `model.recommend` мы передаём идентификатор пользователя, который получаем обратным преобразованием из категории, транспонированную матрицу взаимодействий, число необходимых рекомендаций и список элементов, которые мы договорились фильтровать из ответа.

Возвращает метод список пар (`element_cat`, `score`), отсортированный по вторым элементам. Из него необходимо достать все первые элементы пар и из категории преобразовать их к `element_uid`.

In [38]:
result = {}

user_uid_to_cat = dict(zip(
    transactions['user_uid'].cat.categories,
    range(len(transactions['user_uid'].cat.categories))
))
element_uid_to_cat = dict(zip(
    transactions['element_uid'].cat.categories,
    range(len(transactions['element_uid'].cat.categories))
))
filtered_elements_cat = {k: [element_uid_to_cat.get(x, None) for x in v] for k, v in filtered_elements.items()}

for user_uid in tqdm.tqdm_notebook(test_users):
    # transform user_uid to model's internal user category
    try:
        user_cat = user_uid_to_cat[user_uid]
    except LookupError:
        continue
    
    # perform inference
    recs = model_transactions.recommend(
        user_cat,
        transactions_matrix_T,
        N=20,
        filter_already_liked_items=True,
        filter_items=filtered_elements_cat.get(user_uid, set())
    )
    
    # drop scores and transform model's internal elelemnt category to element_uid for every prediction
    # also convert np.uint64 to int so it could be json serialized later
    result[user_uid] = [int(transactions['element_uid'].cat.categories[i]) for i, _ in recs]

**Важно:** Не все тестовые пользователи есть в `ratings.csv` и не все из них есть в `transactions.csv`. Используя только один источник данных мы не можем построить полное предсказание. Такой ответ с неполным числом пользователей бдет принят системой, но при вычислении средней метрики метрика для отсутствующих пользователей будет принята равной нулю.

In [39]:
# result = {}
user_uid_to_cat = dict(zip(
    ratings['user_uid'].cat.categories,
    range(len(ratings['user_uid'].cat.categories))
))
element_uid_to_cat = dict(zip(
    ratings['element_uid'].cat.categories,
    range(len(ratings['element_uid'].cat.categories))
))
filtered_elements_cat = {k: [element_uid_to_cat.get(x, None) for x in v] for k, v in filtered_elements.items()}

for user_uid in tqdm.tqdm_notebook(test_users):
    # transform user_uid to model's internal user category
    try:
        user_cat = user_uid_to_cat[user_uid]
    except LookupError:
        continue
    
    # perform inference
    recs = model.recommend(
        user_cat,
        ratings_matrix_T,
        N=20,
        filter_already_liked_items=True,
        filter_items=filtered_elements_cat.get(user_uid, set())
    )
    
    # drop scores and transform model's internal elelemnt category to element_uid for every prediction
    # also convert np.uint64 to int so it could be json serialized later
    result[user_uid] = [int(ratings['element_uid'].cat.categories[i]) for i, _ in recs]

In [43]:
# result = {}
user_uid_to_cat = dict(zip(
    bookmarks['user_uid'].cat.categories,
    range(len(bookmarks['user_uid'].cat.categories))
))
element_uid_to_cat = dict(zip(
    bookmarks['element_uid'].cat.categories,
    range(len(bookmarks['element_uid'].cat.categories))
))
filtered_elements_cat = {k: [element_uid_to_cat.get(x, None) for x in v] for k, v in filtered_elements.items()}

for user_uid in tqdm.tqdm_notebook(test_users):
    # transform user_uid to model's internal user category
    try:
        user_cat = user_uid_to_cat[user_uid]
    except LookupError:
        continue
    
    # perform inference
    recs = model_bookmarks.recommend(
        user_cat,
        bookmarks_matrix_T,
        N=20,
        filter_already_liked_items=True,
        filter_items=filtered_elements_cat.get(user_uid, set())
    )
    
    # drop scores and transform model's internal elelemnt category to element_uid for every prediction
    # also convert np.uint64 to int so it could be json serialized later
    result[user_uid] = [int(bookmarks['element_uid'].cat.categories[i]) for i, _ in recs]

In [46]:
len(result)

50000

In [45]:
with open('answer.json', 'w') as f:
    json.dump(result, f)